# Предсказание региона нефтедобычи с максимальной прибыльностью

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль.


## План выполнения работы:
### <a href=#1>1. Загрузка и подготовка данных</a>

### <a href=#2>2. Обучение и проверка модели.</a>
-    <a href=#2_1> 2.1 Разбиение данных на обучающую и валидационную выборки в соотношении 75:25.</a>
-    <a href=#2_2> 2.2 Обучение модели.</a>
-    <a href=#2_4> 2.4 Выводы.</a>

### <a href=#3>3. Подготовка к расчёту прибыли:</a>
-    <a href=#3_1> 3.1 Рассчёт достаточного объёма сырья для безубыточной разработки новой скважины.</a>
-    <a href=#3_2> 3.2 Выводы.</a>

### <a href=#4>4. Расчёт рисков и прибыли</a>
-    <a href=#4_1> 4.1 Расчёты для каждого региона.</a>

### <a href=#5>5. Вывод.</a>   


# <a id='1'> 1. Загрузка и подготовка данных</a> 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
from sklearn import metrics
from scipy import stats as st

In [2]:
geo_data_0 = pd.read_csv('\practikum\geo_data_0.csv')
geo_data_1 = pd.read_csv('\practikum\geo_data_1.csv')
geo_data_2 = pd.read_csv('\practikum\geo_data_2.csv')

In [3]:
datas = {'регион 0':geo_data_0, 
         'регион 1':geo_data_1, 
         'регион 2':geo_data_2}

In [4]:
for key in datas.keys():
    print('Содержание тaблицы '+key)
    display(datas[key].head(3))

Содержание тaблицы регион 0


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647


Содержание тaблицы регион 1


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305


Содержание тaблицы регион 2


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910


In [5]:
for key in datas.keys():
    print('Информация о таблице '+key)
    display(datas[key].info())

Информация о таблице регион 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

Информация о таблице регион 1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

Информация о таблице регион 2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

#### Вывод:
Целевой признак находится в столбцах product. Признаки по которым необходимо построить модель находятся в столбцах f0, f1 и f2. Нет необходимости нормализовать их. Признак id не будет участвовать в обучении.

Запишим целевую переменную и признаки в разные файлы:

In [6]:
def get_train_test(data):
    features = data.drop(['id', 'product'], axis=1)
    target = data['product']
    print('Размеры матрицы признаков:', features.shape)
    print('Размер вектора целевого признака:', target.shape)
    return features, target

In [7]:
features_0, target_0 = get_train_test(geo_data_0)

Размеры матрицы признаков: (100000, 3)
Размер вектора целевого признака: (100000,)


In [8]:
features_1, target_1 = get_train_test(geo_data_1)

Размеры матрицы признаков: (100000, 3)
Размер вектора целевого признака: (100000,)


In [9]:
features_2, target_2 = get_train_test(geo_data_2)

Размеры матрицы признаков: (100000, 3)
Размер вектора целевого признака: (100000,)


# <a id='2'> 2. Обучение и проверка модели</a> 

## <a id='2_1'> 2.1 Разбиение данных на обучающую и валидационную выборки</a> 

Разобьём данные в каждой выборке на тренировочную и валидационную части:

In [10]:
def get_train_test(features, target):
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=123)
    print('Размеры обучающей выборки', features_train.shape, target_train.shape)
    print('Размеры валидационной выборки', features_valid.shape, target_valid.shape)
    return features_train, features_valid, target_train, target_valid

In [11]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = get_train_test(features_0, target_0)

Размеры обучающей выборки (75000, 3) (75000,)
Размеры валидационной выборки (25000, 3) (25000,)


In [12]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = get_train_test(features_1, target_1)

Размеры обучающей выборки (75000, 3) (75000,)
Размеры валидационной выборки (25000, 3) (25000,)


In [13]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = get_train_test(features_2, target_2)

Размеры обучающей выборки (75000, 3) (75000,)
Размеры валидационной выборки (25000, 3) (25000,)


## <a id='2_2'> 2.2 Обучение модели.</a> 

In [14]:
def AverageStock(features_train, target_train, features_valid, target_valid):
    linr_model = LinearRegression().fit(features_train, target_train)
    pred = linr_model.predict(features_valid)
    rmse = np.sqrt(metrics.mean_squared_error(target_valid, pred))
    print('Средний запас предсказанного сырья:', pred.mean(), 'тыс. баррелей')
    print('Средняя квадратическая ошибка модели:', rmse)
    return pred

In [15]:
print('Данные для региона 0')
predict_0 = AverageStock(features_train_0, target_train_0 ,features_valid_0, target_valid_0)

Данные для региона 0
Средний запас предсказанного сырья: 92.54936189116309 тыс. баррелей
Средняя квадратическая ошибка модели: 37.64786282376177


In [16]:
print('Данные для региона 1')
predict_1 = AverageStock(features_train_1, target_train_1 ,features_valid_1, target_valid_1)

Данные для региона 1
Средний запас предсказанного сырья: 69.28001860653976 тыс. баррелей
Средняя квадратическая ошибка модели: 0.8954139804944297


In [17]:
print('Данные для региона 2')
predict_2 = AverageStock(features_train_2, target_train_2 ,features_valid_2, target_valid_2)

Данные для региона 2
Средний запас предсказанного сырья: 95.09859933591373 тыс. баррелей
Средняя квадратическая ошибка модели: 40.12803006598514


## <a id='2_3'> 2.3 Выводы.</a> 

Самое большое значение предсказанного запаса сырья у региона №2. Оно составляет <b>95.099</b> тыс. баррелей. Средняя квадратическая ошибка для этого региона тоже самая большая <b>40.128</b> тыс. баррелей.

Самое маленькое значение ошибки у региона №1 (<b>0.895</b>) при среднем значении предсказанного запаса сырья <b>69.280</b> тыс. баррелей.

# <a id='3'> 3. Подготовка к расчёту прибыли</a> 

In [18]:
BUDGET = 10000000000 #Бюджет на разработку скважин в регионе
UNIT_INCOME = 450000 #Доход с каждой единицы продукта (тыс. баррелей сырья)
NUMBER_OF_WELLS = 500 #Количество скважин, исследуемых при разведне региона
NUMBER_OF_BEST_WELLS = 200 #Лучшие скважины для расчёта прибыли.

## <a id='3_1'> 3.1 Рассчёт достаточного объёма сырья для безубыточной разработки новой скважины.</a> 

In [19]:
print('Объём сырья, достаточный для безубыточной разработки новой скважины: {:.3f} тыс. баррелей'.format(
    (BUDGET/UNIT_INCOME)/NUMBER_OF_BEST_WELLS))

Объём сырья, достаточный для безубыточной разработки новой скважины: 111.111 тыс. баррелей


## <a id='3_2'> 3.2 Выводы.</a> 

При бюджете в 10 млрд рублей на разработку скважин в выбранном регионе и доходе от единицы сырья в 450 тыс. рублей, объём сырья, добываемый с каждой из 200 лучших скважин из 500 исследуемых в данном регионе должен быть больше 111.111 тыс. баррелей для выполнения условия безубыточной разработки. 

Из трёх регионов среднее значение предсказанного запаса сырья у региона №2 близко к рассчитаному значению и состаляет 95.099 тыс. баррелей. У региона №0 это значение равно 92.549 тыс. баррелей.

# <a id='4'> 4. Расчёт рисков и прибыли.</a> 

Напишим функцию для автоматического расчёта потенциальной прибыли от региона.

In [20]:
def profit(target, predict, count=200):
    best_pred = pd.DataFrame(data=target, index=target.index)#таблица с известными значениями объёмов 
    best_pred['predict'] = predict #колонка с предсказанными значениями по этим скважинам

    best_number = best_pred.sort_values(by='predict', ascending=False).head(count) #лучшие по предсказаниям
    target_of_best = best_number['product'].sum()
    
    return (target_of_best*UNIT_INCOME)-BUDGET

## <a id='4_1'>4.1 Расчёты для каждого региона.</a> 

In [21]:
def get_profit(target, predict, number):
    pred_df = pd.DataFrame(data=target, index=target.index)
    pred_df['predict'] = predict
    state = np.random.RandomState(123)
    values = []
    risk = []
    for i in range(1000):
        best_pred = pred_df.sample(n=number, replace=True, random_state=state)
        target_subsample = best_pred['product']
        pred_subsample = best_pred['predict']
        
        values.append(profit(target_subsample, pred_subsample, count=200))
        
    
    values = pd.Series(values)
    confidence_interval = st.t.interval(0.95, len(values)-1, loc=values.mean(), scale=values.sem())
    losses = values.min()
    risk = (values<0).sum()/1000
    
    mean = values.mean()
    print("Средняя прибыль составит: {:.3f} млрд рублей".format(mean/1000000000))
    print("95%-й доверительный интервал: ", confidence_interval)
    print('Потенциальные убытки:{:.3f} млрд рублей'.format(losses/1000000000))
    print('Риск убытков: {:.2%}'.format(risk))


In [22]:
print('Для региона №0 потенциальная прибыль составит:')
get_profit(target_valid_0, predict_0, 500)

Для региона №0 потенциальная прибыль составит:
Средняя прибыль составит: 0.477 млрд рублей
95%-й доверительный интервал:  (461032015.40435, 493801633.1284743)
Потенциальные убытки:-0.254 млрд рублей
Риск убытков: 4.10%


In [23]:
print('Для региона №1 потенциальная прибыль составит:')
get_profit(target_valid_1, predict_1, 500)

Для региона №1 потенциальная прибыль составит:
Средняя прибыль составит: 0.479 млрд рублей
95%-й доверительный интервал:  (466724922.34201944, 491655400.2586471)
Потенциальные убытки:-0.253 млрд рублей
Риск убытков: 0.90%


In [24]:
print('Для региона №2 потенциальная прибыль составит:')
get_profit(target_valid_2, predict_2, 500)

Для региона №2 потенциальная прибыль составит:
Средняя прибыль составит: 0.343 млрд рублей
95%-й доверительный интервал:  (326730862.64534986, 360177890.51640105)
Потенциальные убытки:-0.530 млрд рублей
Риск убытков: 9.90%


# <a id='4'> 5. Вывод.</a> 

Самое большое значение средней прибыли наблюдается для региона №1 0.479 млрд рублей. У него же самая низкая вероятность потерпеть убытки 0.90%. Он является наиболее подходящим для дальнейшей работы.

Самые плохие показатели у региона №2. Средняя прибыль составит: 0.343 млрд рублей, а риск убытков: 9.90%. Этот район манее всего подходит для разработок.
